In [ ]:
%load_ext autoreload
%autoreload 2

**Two steps :** 
- create embeding and save 
- create retrier and generation

In [1]:
%load_ext autoreload
%autoreload 2


from src.vector_store import VectorStore
from src.config import Config


# document_loader.py
import bs4
from langchain_community.document_loaders import WebBaseLoader

class DocumentLoader:
    @staticmethod
    def load_web_documents(web_paths):
        bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
        loader = WebBaseLoader(
            web_paths=web_paths,
            bs_kwargs={"parse_only": bs4_strainer},
        )
        return loader.load()
    
# document_processor.py
from langchain_text_splitters import RecursiveCharacterTextSplitter

class DocumentProcessor:
    def __init__(self, chunk_size=2000, chunk_overlap=200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap, 
            add_start_index=True
        )

    def split_documents(self, docs):
        return self.text_splitter.split_documents(docs)
    

# llm.py
from langchain_openai import ChatOpenAI


class LLM:
    def __init__(self,config=None):
        self.config = config if config else Config()
        self.model = ChatOpenAI(model=self.config.MODEL_NAME)


USER_AGENT environment variable not set, consider setting it to identify your requests.


USER_AGENT environment variable not set, consider setting it to identify your requests.


**First Part**

In [58]:
%load_ext autoreload
%autoreload 2


from src.config import Config
from src.document import DocumentLoader,DocumentProcessor
from src.vector_store import VectorStore
from src.llm import LLM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
config = Config()
config

Config()

In [60]:
# Load documents
web_paths = [
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-01-10-inference-optimization/",
    "https://lilianweng.github.io/posts/2022-06-09-vlm/"
]
docs = DocumentLoader.load_web_documents(web_paths)


# # Process documents
processor = DocumentProcessor()

split_docs = processor.split_documents(docs)
# Create vector store
vector_store = VectorStore()


INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collections?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"


In [61]:
vector_store.add_documents(split_docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/pre-flight-checks "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collections/0c748cc5-3d4e-4701-84c8-d693ac54f10b/upsert "HTTP/1.1 200 OK"


**Second Part**

In [2]:
from src.rag import RAG
from src.config import Config
from src.vector_store import VectorStore
from src.llm import LLM

In [3]:
 # Initialize vector store
vector_store = VectorStore()
retriever = vector_store.get_retriever()

# Initialize LLM
llm = LLM().model

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collections?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"


In [4]:
rag = RAG(retriever,llm)

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
INFO:src.db:Database initialized successfully
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [8]:
rag.run("What is the capital of France?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collections/0c748cc5-3d4e-4701-84c8-d693ac54f10b/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.db:Conversation e8e72cc9-b39c-4aad-b692-dfb725eb21b4 saved successfully
INFO:src.rag:RAG process completed for question: What is the capital of France?...


('The capital of France is Paris.',
 {'answer': 'The capital of France is Paris.',
  'model_used': 'gpt-4o-mini',
  'response_time': 1.646196,
  'relevance': 'RELEVANT',
  'prompt_tokens': 7,
  'completion_tokens': 7,
  'total_tokens': 14,
  'openai_cost': 2.1e-06,
  'question_trend': 0.0},
 'e8e72cc9-b39c-4aad-b692-dfb725eb21b4')

In [3]:
# db.py
import os
import psycopg2
from psycopg2.extras import DictCursor
from datetime import datetime
from zoneinfo import ZoneInfo
import logging

tz = ZoneInfo("Europe/Berlin")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_db_connection():
    try:
        return psycopg2.connect(
            host=os.getenv("POSTGRES_HOST", "postgres"),
            database=os.getenv("POSTGRES_DB", "course_assistant"),
            user=os.getenv("POSTGRES_USER", "your_username"),
            password=os.getenv("POSTGRES_PASSWORD", "your_password"),
        )
    except psycopg2.Error as e:
        logger.error(f"Unable to connect to the database: {e}")
        raise

def init_db():
    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS conversations (
                    id TEXT PRIMARY KEY,
                    question TEXT NOT NULL,
                    answer TEXT NOT NULL,
                    model_used TEXT NOT NULL,
                    response_time FLOAT NOT NULL,
                    relevance TEXT,
                    prompt_tokens INTEGER,
                    completion_tokens INTEGER,
                    total_tokens INTEGER,
                    openai_cost FLOAT,
                    question_trend FLOAT,
                    timestamp TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
                )
            """)
        conn.commit()
        logger.info("Database initialized successfully")
    except psycopg2.Error as e:
        logger.error(f"Error initializing database: {e}")
        conn.rollback()
    finally:
        conn.close()

def save_conversation(conversation_id, question, answer_data):
    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute("""
                INSERT INTO conversations 
                (id, question, answer, model_used, response_time, relevance, 
                prompt_tokens, completion_tokens, total_tokens, openai_cost, question_trend)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                conversation_id,
                question,
                answer_data['answer'],
                answer_data['model_used'],
                answer_data['response_time'],
                answer_data['relevance'],
                answer_data['prompt_tokens'],
                answer_data['completion_tokens'],
                answer_data['total_tokens'],
                answer_data['openai_cost'],
                answer_data["question_trend"]
            ))
        conn.commit()
        logger.info(f"Conversation {conversation_id} saved successfully")
    except psycopg2.Error as e:
        logger.error(f"Error saving conversation: {e}")
        conn.rollback()
    finally:
        conn.close()

def get_recent_conversations(limit=5):
    conn = get_db_connection()
    try:
        with conn.cursor(cursor_factory=DictCursor) as cur:
            cur.execute("""
                SELECT * FROM conversations
                ORDER BY timestamp DESC
                LIMIT %s
            """, (limit,))
            return cur.fetchall()
    except psycopg2.Error as e:
        logger.error(f"Error retrieving recent conversations: {e}")
        return []
    finally:
        conn.close()

def drop_db():
    conn = get_db_connection()
    try:
        with conn.cursor() as cur:
            cur.execute("DROP TABLE IF EXISTS feedback")
            cur.execute("DROP TABLE IF EXISTS conversations")
        conn.commit()
        logger.info("Database dropped successfully")
    except psycopg2.Error as e:
        logger.error(f"Error dropping database: {e}")
        conn.rollback()
    finally:
        conn.close()


#drop_db()
init_db()

INFO:__main__:Database initialized successfully


In [5]:
import uuid
from datetime import datetime
import logging
import numpy as np
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# from db import save_conversation, get_db_connection, init_db
from collections import deque
from src.embedding import get_embedding,EmbeddingFactory
from src.llm import LLM

In [9]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class RAG:
    def __init__(self, retriever, llm, db_connection=None,last_question_count=20,embedding_type="SentenceTransformer"):
        self.retriever = retriever
        self.llm = llm
        self.prompt = hub.pull("rlm/rag-prompt")
        self.db_connection = db_connection
        self.initialize_db()
        self.question_queue = deque([],last_question_count)
        self.embedding_model = EmbeddingFactory().get_embedding_model(embedding_type)

    def initialize_db(self):
        init_db()
        if self.db_connection is None:
            self.db_connection = get_db_connection()

    @staticmethod
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def create_chain(self):
        return (
            {"context": self.retriever | self.format_docs, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )

    def run(self, question):
        try:
            chain = self.create_chain()
            start_time = datetime.now()
            answer = chain.invoke(question)
            end_time = datetime.now()
            response_time = (end_time - start_time).total_seconds()

            self.question_queue.append(self.embedding_model.embed(question))

    
            # Prepare answer data
            answer_data = {
                "answer": answer,
                "model_used": self.llm.model_name,
                "response_time": response_time,
                "relevance": self.evaluate_relevance(question, answer),
                "prompt_tokens": self.llm.get_num_tokens(question),
                "completion_tokens": self.llm.get_num_tokens(answer),
                "total_tokens": self.llm.get_num_tokens(question) + self.llm.get_num_tokens(answer),
                "openai_cost": self.calculate_cost(self.llm.get_num_tokens(question), self.llm.get_num_tokens(answer)),
                "question_trend" : self.analyze_question_trend()
            }

            # Save conversation
            conversation_id = str(uuid.uuid4())
            save_conversation(conversation_id, question, answer_data)

            logger.info(f"RAG process completed for question: {question[:50]}...")
            return answer, answer_data, conversation_id

        except Exception as e:
            logger.error(f"Error in RAG process: {e}")
            raise

    def evaluate_relevance(self, question, answer):
        # Implement relevance evaluation logic here
        # This could be a simple heuristic or a more complex model
        return "RELEVANT"  # Placeholder

    def calculate_cost(self, prompt_tokens, completion_tokens):
        # Implement cost calculation based on your pricing model
        return (prompt_tokens * 0.0001 + completion_tokens * 0.0002) / 1000  # Example calculation

    def get_recent_conversations(self, limit=5):
        # This method could be used to retrieve recent conversations from the database
        from rag.src.db import get_recent_conversations
        return get_recent_conversations(limit)
    
    def analyze_question_trend(self, window_size=5):
        """
        Calculate trend based on the last few question embeddings.
        """
        if len(self.question_queue) < window_size:
            logger.warning("Not enough questions for trend calculation")
            return 0.0 # Return zero vector if not enough data

        recent_embeddings = list(self.question_queue)[-window_size:]
        trend = np.mean(recent_embeddings, axis=0)
        return float(np.mean(trend))
    
    
    

In [10]:
#from db import get_db_connection
from dotenv import load_dotenv
from src.vector_store import VectorStore
from src.llm import LLM

load_dotenv()

db_connection = get_db_connection()
retriever =  VectorStore().get_retriever()


# Initialize LLM
llm = LLM().model
rag = RAG(retriever, llm, db_connection)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collections?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"
/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
INFO:__main__:Database initialized successfully
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-

In [22]:
import time

In [ ]:

for i in range(100):
    print(rag.run("what is the meaning of life?"))
    time.sleep(3)
#answer, answer_data,conversation_id = rag.run("what is the meaning of life?")

In [12]:
import chromadb
from chromadb.config import Settings
from chromadb import Client, HttpClient
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)

In [14]:
Client(
    Settings(
        chroma_server_host='localhost',
        chroma_server_http_port=8000
    )
)

In [38]:
from chromadb import Client,HttpClient
from chromadb.config import Settings
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from uuid import uuid4

In [41]:
settings = Settings(
    chroma_server_host= "locahost",
    chroma_server_http_port= 8000
)

embeddings = OpenAIEmbeddings()

In [42]:
client = Client(settings)

In [56]:
vector_store = Chroma(
    client=client,
    collection_name="test",
    embedding_function=embeddings,

)

In [57]:
document_0 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_1 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_0,
    document_1,
]
  
  
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids,collection_name="test")

['b54332c7-cfba-4f90-8d50-b946d50278b9',
 'bea73e38-f222-41ba-b4d5-a84352e87216']

In [58]:
collection_name = client.get_collection("test")

In [59]:
collection_name.get(ids=['b54332c7-cfba-4f90-8d50-b946d50278b9',
 'bea73e38-f222-41ba-b4d5-a84352e87216'])

{'ids': ['b54332c7-cfba-4f90-8d50-b946d50278b9',
  'bea73e38-f222-41ba-b4d5-a84352e87216'],
 'embeddings': None,
 'documents': ['The stock market is down 500 points today due to fears of a recession.',
  'I have a bad feeling I am going to get deleted :('],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'news'}, {'source': 'tweet'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [3]:


class RAG:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.prompt = hub.pull("rlm/rag-prompt")

    def create_embeddings(self, texts):
        pass

    def storage_embeddings(self, embeddings):
        pass

    def creat_retriever(self, embeddings):
        pass

    def create_rag_chain(self):
        return (
            {"context": self.retriever | self.format_docs, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
    
    def run(self, question):
        chain = self.create_rag_chain()
        return chain.invoke(question)

    @staticmethod
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)


### Workspace

In [1]:
%load_ext autoreload
%autoreload 2


from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [44]:
# load
import bs4
from langchain_community.document_loaders import WebBaseLoader



bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))



web_paths = [
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-01-10-inference-optimization/",
    "https://lilianweng.github.io/posts/2022-06-09-vlm/"

]

loader = WebBaseLoader(
    web_paths=web_paths,
    bs_kwargs={"parse_only": bs4_strainer},
)

docs = loader.load()

In [52]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

170

In [74]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(),persist_directory="./chroma_db")

In [75]:
all_splits[5]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'start_index': 6777}, page_content='Some interesting observations on model hallucination behavior:\n\nError rates are higher for rarer entities in the task of biography generation.\nError rates are higher for facts mentioned later in the generation.\nUsing retrieval to ground the model generation significantly helps reduce hallucination.\n\nWei et al. (2024) proposed an evaluation method for checking long-form factuality in LLMs, named SAFE (Search-Augmented Factuality Evaluator; code). The main difference compared to FActScore is that for each self-contained, atomic fact, SAFE uses a language model as an agent to iteratively issue Google Search queries in a multi-step process and reason about whether the search results support or do not support the fact. In each step, the agent generates a search query based on a given fact to check, as well as previously obtained search results. After a numbe

In [64]:
question = "What is hallucination in LLM ?"

In [66]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [67]:
retrieved_docs = retriever.invoke(question)


In [76]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [77]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/home/tuvis/miniconda3/envs/hf/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [87]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
    
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    # | llm
    # | StrOutputParser()
)

res = rag_chain.invoke(question)

In [ ]:
print(res.messages[0].content)